In [3]:
import os
from pathlib import Path
import torchio as tio
import nibabel as nib
import numpy as np
import glob
import pandas as pd

Skull stripping with HD-BET

In [ ]:
input_dir = "/Volumes/Extreme_SSD/payam_data/ukbb/orig"
bet_dir = "/Volumes/Extreme_SSD/payam_data/ukbb/bet"
os.makedirs(bet_dir, exist_ok=True)

nii_files = glob.glob(os.path.join(input_dir, "*.nii.gz"))

print(f"Found {len(nii_files)} files.")



In [ ]:
!nohup hd-bet -i /home/ubuntu/volume/Tinception/VBM -o . --save_bet_mask > hdbet.log 2>&1 &

Preprocesing with Torch IO

In [30]:
df = pd.read_csv("/Users/payamsadeghishabestari/tinception/material/with_qc/behavioural/tinception_master_ready_for_matching_qc.csv")

tio_subjects = []
subjects = []
bet_dir = Path("/Users/payamsadeghishabestari/temp_folder/bet")
for fname in bet_dir.iterdir():
    if fname.suffix == ".gz" and "_bet" not in fname.stem:
        subjects.append(fname.stem[:-4])

for sid in subjects:
    row = df.loc[df['subject ID'] == sid].iloc[0]

    subject = tio.Subject(
                        t1=tio.ScalarImage(bet_dir / f"{sid}.nii.gz"),
                        brain_mask=tio.LabelMap(bet_dir / f"{sid}_bet.nii.gz"),
                        sex = row['sex'],
                        age = row['age'],
                        site = row['site'],
                        PTA = row['PTA'],
                        label = row['group']
                        )
    tio_subjects.append(subject)

preprocess = tio.Compose([
                        tio.ToCanonical(),                   
                        tio.Resample(1),                     
                        tio.ZNormalization(masking_method='brain_mask'),  
                        tio.CropOrPad((160, 192, 160))      
                        ])

dataset = tio.SubjectsDataset(tio_subjects, transform=preprocess)

In [31]:
dataset